#  <center>        **Segmenting and Clustering Neighborhoods in Toronto**   

This notebook consists of all the three parts of the assignment





### **Importing requisite libraries**

In [38]:


from bs4 import BeautifulSoup 

import numpy as np 

import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json 

from geopy.geocoders import Nominatim 

import requests 
from pandas.io.json import json_normalize 

import matplotlib.cm as cm
import matplotlib.colors as colors


from sklearn.cluster import KMeans

!pip install folium
import folium 

!pip install geocoder
import geocoder 

## Assignment Part-1  Obtaining Dataframe of postal codes in Toronto, Canada
  The table of postal codes is scaraped using Beautiful Soup package from Wikipedia Page 

Scraping data from Wikipedia and storing it in a Dataframe

In [39]:
# Get webpage
url = 'https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=945633050.'
page = requests.get(url).text

# Extract the table
soup = BeautifulSoup(page, 'html.parser')
table = soup.find('table', class_='sortable')

# Get the values from the table and store  it into a dataframe
row = [] 

for tr in table.find_all('tr'):                                          
    if tr.find_all('th') == []:                                           
        row.append([td.get_text(strip=True) for td in tr.find_all('td')]) 

column_names = ['Postcode', 'Borough', 'Neighborhood']

neighborhoods_raw = pd.DataFrame(row, columns=column_names) 
neighborhoods_raw.head(10)

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
8,M8A,Not assigned,Not assigned
9,M9A,Etobicoke,Islington Avenue


**Processing Data frame**


1. Rows having boroughs as 'Not assigned' values  are dropped.
2. Group neighborhoods in the same borough
3. Assigning borough names to neighborhoods where neighborhood is listed as 'Not   assigned'  
 



In [0]:
drop_index = neighborhoods_raw[neighborhoods_raw['Borough'] == 'Not assigned'].index 
neighborhoods = neighborhoods_raw.drop(drop_index, axis=0)                         
neighborhoods.reset_index(drop=True, inplace=True)                                   

In [0]:
neighborhoods = neighborhoods.groupby(['Postcode', 'Borough'], as_index=False).agg(lambda x: ', '.join(x))


In [44]:
nh_na = neighborhoods[neighborhoods['Neighborhood'] == 'Not assigned'].index # index of those rows
neighborhoods.iloc[nh_na, 2] = neighborhoods['Borough'][nh_na]
neighborhoods

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [45]:
neighborhoods.shape

(103, 3)

## Assignment Part-2  Determiniation of Latitude and Longitude
In order to utilize the Foursquare location data the latitude and the longitude coordinates of each neighborhood is obtained using Geocoder. 


In [0]:

lat = []
lng = []
lat_lng_coordinates = None


postal_code = neighborhoods['Postcode']

for pc in postal_code:
    g = geocoder.arcgis('{}, Toronto, Ontario'.format(pc))
    lat_lng_coordinates = g.latlng
    lat.append(lat_lng_coordinates[0])
    lng.append(lat_lng_coordinates[1])

In [47]:
nh_complete = neighborhoods
nh_complete['Latitude'] = lat
nh_complete['Longitude'] = lng
nh_complete

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.811525,-79.195517
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.785665,-79.158725
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.765815,-79.175193
3,M1G,Scarborough,Woburn,43.768369,-79.217590
4,M1H,Scarborough,Cedarbrae,43.769688,-79.239440
5,M1J,Scarborough,Scarborough Village,43.743125,-79.231750
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.726276,-79.263625
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.713054,-79.285055
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.724235,-79.227925
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.696770,-79.259967


## Using Foursquare API to cluster and explore Toronto's neighborhoods
Explore and cluster the neighborhoods in Toronto. Analyzing  boroughs that contain the word Toronto

In [0]:

CLIENT_ID = '' #  Foursquare ID
CLIENT_SECRET = 'F' #  Foursquare Secret
VERSION = '20180605' # Foursquare API version


### Removing entries that do not contain "Toronto"


In [55]:
toronto = nh_complete[nh_complete['Borough'].str.find('Toronto') != -1].reset_index(drop=True)
toronto.shape


(39, 5)

## Map of Toronto

In [56]:
g = geocoder.arcgis('Toronto, Ontario')
lat_tor = g.latlng[0]
lng_tor = g.latlng[1]


map_toronto = folium.Map(location=[lat_tor, lng_tor], zoom_start=11)


for lat, lng, bor, postcode in zip(toronto['Latitude'], toronto['Longitude'], toronto['Borough'], toronto['Postcode']):
    label = '{}, {}'.format(postcode, bor)        
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat, lng],
                        radius=5,
                        popup=label,
                        color='blue',
                        fill=True,
                        fill_color='#3186cc',
                        fill_opacity=0.7,
                        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [0]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):        
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
                CLIENT_ID, 
                CLIENT_SECRET, 
                VERSION, 
                lat, 
                lng, 
                radius, 
                LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(name,
                             lat,
                             lng,
                             v['venue']['name'],
                             v['venue']['location']['lat'],
                             v['venue']['location']['lng'],
                             v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Postcode', 
                  'Latitude', 
                  'Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [0]:
# Limit the number of venues to 100
LIMIT = 100

# Get the venues for each postcode
toronto_venues = getNearbyVenues(names=toronto['Postcode'],
                                 latitudes=toronto['Latitude'],
                                 longitudes=toronto['Longitude'],
                                 radius=700) # limit radius to 700 meters

In [59]:
print(toronto_venues.shape)
toronto_venues.head(10)

(2457, 7)


,Postcode,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M4E,43.676531,-79.295425,Glen Manor Ravine,43.676821,-79.293942,Trail
1,M4E,43.676531,-79.295425,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,M4E,43.676531,-79.295425,Mastermind Toys,43.671453,-79.293971,Toy / Game Store
3,M4E,43.676531,-79.295425,Tori's Bakeshop,43.672114,-79.290331,Vegetarian / Vegan Restaurant
4,M4E,43.676531,-79.295425,Beaches Bake Shop,43.680363,-79.289692,Bakery
5,M4E,43.676531,-79.295425,The Beech Tree,43.680493,-79.288846,Gastropub
6,M4E,43.676531,-79.295425,Veloute Bistro,43.672267,-79.289584,French Restaurant
7,M4E,43.676531,-79.295425,Sanna's Farmacia,43.670929,-79.295969,Juice Bar
8,M4E,43.676531,-79.295425,Beacher Cafe,43.671938,-79.291238,Breakfast Spot
9,M4E,43.676531,-79.295425,Shoppers Drug Mart,43.670639,-79.297690,Pharmacy


In [60]:
num_venues = pd.DataFrame()
num_venues['Postcode'] = toronto_venues.groupby('Postcode').count().reset_index()['Postcode']
num_venues['Venue Count'] = toronto_venues.groupby('Postcode').count().reset_index()['Venue']
num_venues

,Postcode,Venue Count
0,M4E,29
1,M4K,31
2,M4L,34
3,M4M,83
4,M4N,7
5,M4P,21
6,M4R,37
7,M4S,43
8,M4T,6
9,M4V,23


In [61]:
# One hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# Add Postcode column
toronto_onehot['Postcode'] = toronto_venues['Postcode'] 

# Move postcode column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_grouped = toronto_onehot.groupby('Postcode').mean().reset_index()
toronto_grouped.head(10)

,Postcode,Accessories Store,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Disc Golf,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kitchen Supply Store,Korean Restaurant,Latin American Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoor Sculpture,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pier,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soup Place,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Track,Trail,Train Station,Tram Station,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,M4E,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.034483,0.034483,0.068966,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.03448

In [0]:
# Define a function that returns the sorted categories
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]                                       # row of the table of interest
    row_categories_sorted = row_categories.sort_values(ascending=False) # sort the mean frequency of occurrences
    
    return row_categories_sorted.index.values[0:num_top_venues]         # return the index (category) of the mena frequency values

In [63]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# Create columns according to number of top venues
columns = ['Postcode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind])) # 1st, 2nd, 3rd
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# Create a new dataframe
toronto_venues_sorted = pd.DataFrame(columns=columns)
toronto_venues_sorted['Postcode'] = toronto_grouped['Postcode']

# Each row of the new dataframe is filled with the sorted categories of each row of toronto_goruped dataframe
for ind in np.arange(toronto_grouped.shape[0]):
    toronto_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

toronto_venues_sorted.head()

,Postcode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,Pizza Place,Bar,Japanese Restaurant,French Restaurant,Bank,Thai Restaurant,Neighborhood,Café,Juice Bar,Gastropub
1,M4K,Greek Restaurant,Ice Cream Shop,Café,Bank,Business Service,Scenic Lookout,Bubble Tea Shop,Brewery,Chinese Restaurant,Bookstore
2,M4L,Café,Brewery,Coffee Shop,Fast Food Restaurant,Bakery,Skate Park,Sushi Restaurant,Steakhouse,Liquor Store,Fish & Chips Shop
3,M4M,Coffee Shop,Café,Bakery,Diner,American Restaurant,Italian Restaurant,Brewery,Pizza Place,Bar,Pub
4,M4N,Bus Line,Park,Restaurant,Coffee Shop,Bookstore,Gym / Fitness Center,Café,Falafel Restaurant,Farm,Doner Restaurant


### Clustering Neighborhoods



In [64]:
# Set number of clusters
kclusters = 3

toronto_grouped_clustering = toronto_grouped.drop('Postcode', 1)

# Run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# Check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0], dtype=int32)

In [65]:
# Add clustering labels
toronto_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto

# Merge toronto_grouped with toronto to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(toronto_venues_sorted.set_index('Postcode'), on='Postcode')
toronto_merged[['Cluster Labels']] = toronto_merged[['Cluster Labels']].astype('int')

toronto_merged.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676531,-79.295425,0,Pizza Place,Bar,Japanese Restaurant,French Restaurant,Bank,Thai Restaurant,Neighborhood,Café,Juice Bar,Gastropub
1,M4K,East Toronto,"The Danforth West, Riverdale",43.683178,-79.355105,0,Greek Restaurant,Ice Cream Shop,Café,Bank,Business Service,Scenic Lookout,Bubble Tea Shop,Brewery,Chinese Restaurant,Bookstore
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.667965,-79.314667,0,Café,Brewery,Coffee Shop,Fast Food Restaurant,Bakery,Skate Park,Sushi Restaurant,Steakhouse,Liquor Store,Fish & Chips Shop
3,M4M,East Toronto,Studio District,43.660629,-79.334855,0,Coffee Shop,Café,Bakery,Diner,American Restaurant,Italian Restaurant,Brewery,Pizza Place,Bar,Pub
4,M4N,Central Toronto,Lawrence Park,43.728420,-79.387133,0,Bus Line,Park,Restaurant,Coffee Shop,Bookstore,Gym / Fitness Center,Café,Falafel Restaurant,Farm,Doner Restaurant


In [66]:
# Create map
map_clusters = folium.Map(location=[lat_tor, lng_tor], zoom_start=11)

# Set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i * x) ** 2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# Add markers to the map
markers_colors = []
for lat, lon, pc, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Postcode'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(pc) + ', ' + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker([lat, lon],
                        radius=5,
                        popup=label,
                        color=rainbow[cluster-1],
                        fill=True,
                        fill_color=rainbow[cluster-1],
                        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [67]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,0,Pizza Place,Bar,Japanese Restaurant,French Restaurant,Bank,Thai Restaurant,Neighborhood,Café,Juice Bar,Gastropub
1,East Toronto,0,Greek Restaurant,Ice Cream Shop,Café,Bank,Business Service,Scenic Lookout,Bubble Tea Shop,Brewery,Chinese Restaurant,Bookstore
2,East Toronto,0,Café,Brewery,Coffee Shop,Fast Food Restaurant,Bakery,Skate Park,Sushi Restaurant,Steakhouse,Liquor Store,Fish & Chips Shop
3,East Toronto,0,Coffee Shop,Café,Bakery,Diner,American Restaurant,Italian Restaurant,Brewery,Pizza Place,Bar,Pub
4,Central Toronto,0,Bus Line,Park,Restaurant,Coffee Shop,Bookstore,Gym / Fitness Center,Café,Falafel Restaurant,Farm,Doner Restaurant
5,Central Toronto,0,Gym,Café,Indian Restaurant,Dessert Shop,Brewery,Department Store,Discount Store,Pharmacy,Food & Drink Shop,Gourmet Shop
6,Central Toronto,0,Clothing Store,Coffee Shop,Café,Restaurant,Sporting Goods Shop,Yoga Studio,Rental Car Location,Thai Restaurant,Italian Restaurant,Juice Bar
7,Central Toronto,0,Dessert Shop,Sandwich Place,Coffee Shop,Pizza Place,Café,Trail,Italian Restaurant,Sushi Restaurant,Gym,American Restaurant
9,Central Toronto,0,Coffee Shop,Pub,Skating Rink,Sushi Restaurant,Yoga Studio,Bank,Fried Chicken Joint,French Restaurant,Light Rail Station,Liquor Store
11,Downtown Toronto,0,Coffee Shop,Restaurant,Pizza Place,Bakery,Gastropub,Flower Shop,Café,Italian Restaurant,Market,Pub


In [68]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Central Toronto,1,Park,Gym,Grocery Store,Thai Restaurant,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant,Elementary School,Dive Bar
10,Downtown Toronto,1,Park,Grocery Store,Candy Store,Playground,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant,Elementary School,Dive Bar


In [69]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Central Toronto,2,Playground,Home Service,Spa,Kitchen Supply Store,Garden,Elementary School,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Event Space


 Most of the neighborhoods  are similar and belong to cluster 0 with the exception of three neighborhoods, M4T and M4W in cluster 1 and M5N in cluster 2.
